In [22]:
from math import pow,fabs
from Archimedean import *
from DHFEs import *
from tqdm import tqdm

### 得分函数

In [23]:
def score(DHFFIBM):
    sum = 0
    for i in DHFFIBM.MD:
        sum = sum+i
    average_MD=sum/len(DHFFIBM.MD)
    sum = 0
    for i in DHFFIBM.NMD:
        sum = sum+i
    average_NMD=sum/len(DHFFIBM.NMD)
    score = average_MD-average_NMD
    return score

### 对偶犹豫模糊距离函数和支持度

In [24]:
## 任意两个对偶犹豫模糊元素的距离

def DHFEs_distance(d1,d2,q,lam):
    # d1 表示第一个对偶犹豫模糊元素，d2 表示第二个犹豫模糊元素
    # q 表示 q 级正交模糊，q=1 为对偶犹豫直觉模糊，q=2 为对偶犹豫毕达哥拉斯模糊，q=3 位对偶犹豫费马模糊
    # lam 表示广义对偶犹豫模糊元素距离参数计算，lam=1 为 Hamming distance，lam=2 为 Euclidean distance.
    # 该距离为对偶犹豫费马模糊元素距离公式
    
    # 注意：后续还需加入隶属度与非隶属度的排序
    
    ## 标准化，若 len(d1.MD)≠len(d2.MD) 则将隶属度的值数量匹配相等，采用乐观匹配 
    lmd = len(d1.MD)-len(d2.MD)
    lnmd = len(d1.NMD)-len(d2.NMD)
    i=0
    if lmd>0:
        while i<lmd:
            d2.MD.append(max(d2.MD))
            i += 1
    else:
        while i<(-lmd):
            d1.MD.append(max(d1.MD))
            i += 1
    j=0
    if lnmd>0:
        while j<lnmd:
            d2.NMD.append(max(d2.NMD))
            j += 1
    else:
        while j<(-lnmd):
            d1.NMD.append(max(d1.NMD))
            j += 1
    # print(d1.MD,d1.NMD)
    # print(d2.MD,d2.NMD)
    # print(len(d1.MD),len(d1.NMD))
    
    mds,nmds = 0,0
    m = 1/(len(d1.MD)+len(d1.NMD))
    
    for x in range(len(d1.MD)):
        mds += pow(fabs(pow(d1.MD[x],q)-pow(d2.MD[x],q)),lam)
    for y in range(len(d1.NMD)):
        nmds += pow(fabs(pow(d1.NMD[y],q)-pow(d2.NMD[y],q)),lam)
    return pow(m*(mds+nmds),1/lam)


## 对偶犹豫模糊元素支持度
def DHFEs_support(d1,d2,q,lam):
    # d1 表示第一个对偶犹豫模糊元素,d2 表示第二个犹豫模糊元素,该函数表示 d1 对 d2 的支持度
    # q 表示对偶犹豫 q 级正交模糊
    # lam 表示广义对偶犹豫模糊元素距离参数计算，lam=1 为 Hamming distance，lam=2 为 Euclidean distance.
    return 1-DHFEs_distance(d1,d2,q,lam)

### 我们提出的算子

In [25]:
## 对偶犹豫费马模糊爱因斯坦加权 Bonferroni Mean 算子(DHFFWBM)

def DHFFEWBM(alter,weight,p,q):
    ## Dual Hesitant Fermatean Fuzzy Einstein Bonferroni Mean Operator
    ## alter means the alternatives
    ## weight means the weighted vector of the attribute
    
    def BonferroniElement(d1,d2,p,q):
        ## DHFF Bonferroni Elements
        s1 = DHFFE_Einstein_Multiply(d1.Einstein_Power(p),d2.Einstein_Power(q))
        s2 = DHFFE_Einstein_Multiply(d1.Einstein_Power(q),d2.Einstein_Power(p))
        return DHFFE_Einstein_Plus(s1,s2)
    
    dlist = []
    if weight == 0:
        ## 权重为 0，说明没有权重，直接将 alter 即 DHFFE 数组赋值给 dl
        dl = alter
        for i,d1 in enumerate(dl):
            for j,d2 in enumerate(dl[:i]):
                dlist.append(BonferroniElement(d1,d2,p,q))
    else:
        ## 将权重向量乘以备选方案列表
        dl = []
        try:
            ## 判断权重向量与数列向量基数是否相同
            assert len(alter) == len(weight)
        except e:
            print("The number of alter is not equal to the number of weight!",e)
        for i in range(len(alter)):
            dl.append(alter[i].Einstein_Times(weight[i]))
        for i,d1 in enumerate(dl):
            for j,d2 in enumerate(dl[:i]):
                dlist.append(BonferroniElement(d1,d2,p,q))
    aggDHFFE = DHFFE([0],[1])
    for agg in dlist:
        aggDHFFE = DHFFE_Einstein_Plus(agg,aggDHFFE)
    aggDHFFE = (aggDHFFE.Einstein_Times(1/(len(alter)*(len(alter)-1)))).Einstein_Power(1/(p+q))    
    return aggDHFFE

### 第一类对比算子

In [ ]:
##########  对偶犹豫费马模糊   #############

In [ ]:
## 对偶犹豫费马模糊加权平均(DHFFWA)

def DHFFWA(alter,weight):
    dlist = []
    try:
        assert len(alter) == len(weight)
    except e:
        print("The number of alter is not equal to the number of weight!",e)
    for i in range(len(alter)):
        dlist.append(alter[i].Algebraic_Times(weight[i]))
    aggDHFFE = DHFFE([0],[1])
    for agg in dlist:
        aggDHFFE = DHFFE_Algebraic_Plus(agg,aggDHFFE)
    return aggDHFFE   

In [ ]:
## 对偶犹豫费马模糊加权几何 (DHFFWG)

def DHFFWG(alter,weight):
    dlist = []
    try:
        assert len(alter) == len(weight)
    except e:
        print("The number of alter is not equal to the number of weight!",e)
    for i in range(len(alter)):
        dlist.append(alter[i].Algebraic_Power(weight[i]))
    aggDHFFE = DHFFE([1],[0])
    for agg in dlist:
        aggDHFFE = DHFFE_Algebraic_Multiply(agg,aggDHFFE)
    return aggDHFFE   

In [ ]:
## 对偶犹豫费马模糊爱因斯坦加权平均 (DHFFEWA)

def DHFFEWA(alter,weight):
    dlist = []
    try:
        assert len(alter) == len(weight)
    except e:
        print("The number of alter is not equal to the number of weight!",e)
    for i in range(len(alter)):
        dlist.append(alter[i].Einstein_Times(weight[i]))
    aggDHFFE = DHFFE([0],[1])
    for agg in dlist:
        aggDHFFE = DHFFE_Einstein_Plus(agg,aggDHFFE)
    return aggDHFFE       

In [ ]:
## 对偶犹豫费马模糊爱因斯坦加权几何 (DHFFEWG)

def DHFFEWG(alter,weight):
    dlist = []
    try:
        assert len(alter) == len(weight)
    except e:
        print("The number of alter is not equal to the number of weight!",e)
    for i in range(len(alter)):
        dlist.append(alter[i].Einstein_Power(weight[i]))
    aggDHFFE = DHFFE([1],[0])
    for agg in dlist:
        aggDHFFE = DHFFE_Einstein_Multiply(agg,aggDHFFE)
    return aggDHFFE       

In [ ]:
##########  对偶犹豫毕达哥拉斯模糊   #############

In [ ]:
## 对偶犹豫毕达哥拉斯模糊加权平均 (DHPFWA)

def DHPFWA(alter,weight):
    dlist = []
    try:
        assert len(alter) == len(weight)
    except e:
        print("The number of alter is not equal to the number of weight!",e)
    for i in range(len(alter)):
        dlist.append(alter[i].Algebraic_Times(weight[i]))
    aggDHPFE = DHPFE([0],[1])
    for agg in dlist:
        aggDHPFE = DHPFE_Algebraic_Plus(agg,aggDHPFE)
    return aggDHPFE       

In [ ]:
## 对偶犹豫毕达哥拉斯模糊加权几何（DHPFWG）

def DHPFWG(alter,weight):
    dlist = []
    try:
        assert len(alter) == len(weight)
    except e:
        print("The number of alter is not equal to the number of weight!",e)
    for i in range(len(alter)):
        dlist.append(alter[i].Algebraic_Power(weight[i]))
    aggDHPFE = DHPFE([1],[0])
    for agg in dlist:
        aggDHPFE = DHPFE_Algebraic_Multiply(agg,aggDHPFE)
    return aggDHPFE   

In [ ]:
## 对偶犹豫毕达哥拉斯模糊爱因斯坦加权平均 (DHPFEWA)

def DHPFEWA(alter,weight):
    dlist = []
    try:
        assert len(alter) == len(weight)
    except e:
        print("The number of alter is not equal to the number of weight!",e)
    for i in range(len(alter)):
        dlist.append(alter[i].Einstein_Times(weight[i]))
    aggDHPFE = DHPFE([0],[1])
    for agg in dlist:
        aggDHPFE = DHPFE_Einstein_Plus(agg,aggDHPFE)
    return aggDHPFE       

In [ ]:
## 对偶犹豫毕达哥拉斯模糊爱因斯坦加权几何 (DHPFEWG)

def DHPFEWG(alter,weight):
    dlist = []
    try:
        assert len(alter) == len(weight)
    except e:
        print("The number of alter is not equal to the number of weight!",e)
    for i in range(len(alter)):
        dlist.append(alter[i].Einstein_Power(weight[i]))
    aggDHPFE = DHPFE([1],[0])
    for agg in dlist:
        aggDHPFE = DHPFE_Einstein_Multiply(agg,aggDHPFE)
    return aggDHPFE       

### 第二类对比算子

In [ ]:
## 对偶犹豫费马模糊加权 Bonferroni Mean (DHFFWBM)

def DHFFWBM(alter,weight,p,q):
    ## Dual Hesitant Fermatean Fuzzy Bonferroni Mean Operator
    ## alter means the alternatives
    ## weight means the weighted vector of the attribute
    
    def BonferroniElement(d1,d2,p,q):
        ## DHFF Bonferroni Elements
        s1 = DHFFE_Algebraic_Multiply(d1.Algebraic_Power(p),d2.Algebraic_Power(q))
        s2 = DHFFE_Algebraic_Multiply(d1.Algebraic_Power(q),d2.Algebraic_Power(p))
        return DHFFE_Algebraic_Plus(s1,s2)
    
    dlist = []
    if weight == 0:
        ## 权重为 0，说明没有权重，直接将 alter 即 DHFFE 数组赋值给 dl
        dl = alter
        for i,d1 in enumerate(dl):
            for j,d2 in enumerate(dl[:i]):
                dlist.append(BonferroniElement(d1,d2,p,q))
    else:
        ## 将权重向量乘以备选方案列表
        dl = []
        try:
            ## 判断权重向量与数列向量基数是否相同
            assert len(alter) == len(weight)
        except e:
            print("The number of alter is not equal to the number of weight!",e)
        for i in range(len(alter)):
            dl.append(alter[i].Algebraic_Times(weight[i]))
        for i,d1 in enumerate(dl):
            for j,d2 in enumerate(dl[:i]):
                dlist.append(BonferroniElement(d1,d2,p,q))
    aggDHFFE = DHFFE([0],[1])
    for agg in dlist:
        aggDHFFE = DHFFE_Algebraic_Plus(agg,aggDHFFE)
    aggDHFFE = (aggDHFFE.Algebraic_Times(1/(len(alter)*(len(alter)-1)))).Algebraic_Power(1/(p+q))    
    return aggDHFFE

In [ ]:
## 对偶犹豫毕达哥拉斯模糊加权 Bonferroni Mean (DHPFWBM)

def DHPFWBM(alter,weight,p,q):
    ## Dual Hesitant Pythagorean Fuzzy Bonferroni Mean Operator
    ## alter means the alternatives
    ## weight means the weighted vector of the attribute
    
    def BonferroniElement(d1,d2,p,q):
        ## DHPF Bonferroni Elements
        s1 = DHPFE_Algebraic_Multiply(d1.Algebraic_Power(p),d2.Algebraic_Power(q))
        s2 = DHPFE_Algebraic_Multiply(d1.Algebraic_Power(q),d2.Algebraic_Power(p))
        return DHPFE_Algebraic_Plus(s1,s2)
    
    dlist = []
    if weight == 0:
        ## 权重为 0，说明没有权重，直接将 alter 即 DHFFE 数组赋值给 dl
        dl = alter
        for i,d1 in enumerate(dl):
            for j,d2 in enumerate(dl[:i]):
                dlist.append(BonferroniElement(d1,d2,p,q))
    else:
        ## 将权重向量乘以备选方案列表
        dl = []
        try:
            ## 判断权重向量与数列向量基数是否相同
            assert len(alter) == len(weight)
        except e:
            print("The number of alter is not equal to the number of weight!",e)
        for i in range(len(alter)):
            dl.append(alter[i].Algebraic_Times(weight[i]))
        for i,d1 in enumerate(dl):
            for j,d2 in enumerate(dl[:i]):
                dlist.append(BonferroniElement(d1,d2,p,q))
    aggDHPFE = DHPFE([0],[1])
    for agg in dlist:
        aggDHPFE = DHPFE_Algebraic_Plus(agg,aggDHPFE)
    aggDHPFE = (aggDHPFE.Algebraic_Times(1/(len(alter)*(len(alter)-1)))).Algebraic_Power(1/(p+q))    
    return aggDHPFE


In [ ]:
# ## 广义对偶犹豫费马模糊加权 Power Average 算子

# def GDHFFWPA(alter,weight,p):
#     # alter 表示要聚合的对偶犹豫费马模糊集
#     # weight 表示对偶犹豫费马模糊元素的权重
#     # p 表示广义 Power Average 算子参数，p=1 则为对偶犹豫费马模糊加权 Power Average 算子
#     dlist = []
#     try:
#         assert len(alter) == len(weight)
#     except e:
#         print("The number of alter is not equal to the number of weight!",e)
        
#     ## 分母
#     s = 0
#     for j in range(len(alter)):
#         ssup = 0
#         for i in range(len(alter)):
#             if i == j:
#                 continue
#             else:
#                 ssup +=  DHFEs_support(alter[j],alter[i],3,2)
#         s += weight[j]*(p + ssup)
    
#     weig = []
#     for i in range(len(alter)):
#         ss = 0
#         for j in range(len(alter)):
#             if i == j:
#                 continue
#             else:
#                 ss += DHFEs_support(alter[i],alter[j],3,2)
#         weig.append(weight[i]*(p+ss)/s)
    
    
#     for i in range(len(alter)):
#         dlist.append(alter[i].Algebraic_Times(weig[i]))
#     aggDHFFE = DHFFE([0],[1])
#     for agg in dlist:
#         aggDHFFE = DHFFE_Algebraic_Plus(agg,aggDHFFE)
#     return aggDHFFE   

In [4]:
## 广义对偶犹豫费马模糊爱因斯坦加权 Power Average 算子

def GDHFFEWPA(alter,weight,p):
    # alter 表示要聚合的对偶犹豫费马模糊集
    # weight 表示对偶犹豫费马模糊元素的权重
    # p 表示广义 Power Average 算子参数，p=1 则为对偶犹豫费马模糊加权 Power Average 算子
    dlist = []
    try:
        assert len(alter) == len(weight)
    except e:
        print("The number of alter is not equal to the number of weight!",e)
        
    al = alter
    ## 分母
    s = 0
    for j in range(len(al)):
        ssup = 0
        for i in range(len(al)):
            if i == j:
                continue
            else:
                ssup +=  DHFEs_support(al[j],al[i],3,2)      ## 使用欧几里得距离
        s += weight[j]*(p + ssup)
    
    weig = []
    for i in range(len(al)):
        ss = 0
        for j in range(len(al)):
            if i == j:
                continue
            else:
                ss += DHFEs_support(al[i],al[j],3,2)        ## 使用欧几里得距离
        weig.append(weight[i]*(p+ss)/s)
    
    
    for i in range(len(alter)):
        dlist.append(alter[i].Einstein_Times(weig[i]))
    aggDHFFE = DHFFE([0],[1])
    for agg in dlist:
        aggDHFFE = DHFFE_Einstein_Plus(agg,aggDHFFE)
    return aggDHFFE   

输入数据和权重

In [5]:
a11 = DHFFE([0.5],[0.4,0.5])
a12 = DHFFE([0.5,0.6],[0.2])
a13 = DHFFE([0.7],[0.1,0.4])
a14 = DHFFE([0.5,0.8],[0.5])
a15 = DHFFE([0.8],[0.6])

a21 = DHFFE([0.9,0.8],[0.3])
a22 = DHFFE([0.6],[0.3])
a23 = DHFFE([0.7,0.8],[0.5])
a24 = DHFFE([0.8],[0.1])
a25 = DHFFE([0.7,0.8],[0.2,0.3])

a31 = DHFFE([0.6,0.7],[0.3])
a32 = DHFFE([0.5,0.7],[0.4])
a33 = DHFFE([0.4,0.6],[0.2,0.5])
a34 = DHFFE([0.8],[0.3,0.4])
a35 = DHFFE([0.7],[0.5])

a41 = DHFFE([0.5],[0.4])
a42 = DHFFE([0.6,0.8],[0.1])
a43 = DHFFE([0.8],[0.2,0.4])
a44 = DHFFE([0.9],[0.2,0.3])
a45 = DHFFE([0.5,0.7],[0.5,0.3])

a51 = DHFFE([0.5,0.8],[0.2])
a52 = DHFFE([0.8],[0.1,0.2])
a53 = DHFFE([0.7,0.8],[0.5])
a54 = DHFFE([0.5],[0.2])
a55 = DHFFE([0.6],[0.2,0.3])

alter1 = [a11,a12,a13,a14,a15]
alter2 = [a21,a22,a23,a24,a25]
alter3 = [a31,a32,a33,a34,a35]
alter4 = [a41,a42,a43,a44,a45]
alter5 = [a51,a52,a53,a54,a55]
weight = [0.2,0.15,0.2,0.3,0.15]

In [26]:
a11 = DHFFE([0.5],[0.4,0.5])
a12 = DHFFE([0.5,0.6],[0.2])
a13 = DHFFE([0.7],[0.1,0.4])
a14 = DHFFE([0.5,0.8],[0.5])
a15 = DHFFE([0.8],[0.6])

alter = [a11,a12,a13,a14,a15]
weight = [0.2,0.15,0.2,0.3,0.15]

In [12]:
al = alter
## 分母
s = 0
for j in range(len(al)):
    ssup = 0
    for i in range(len(al)):
        if i == j:
            continue
        else:
            ssup +=  DHFEs_support(al[j],al[i],3,2)      ## 使用欧几里得距离
    s += weight[j]*(1 + ssup)

weig = []
for i in range(len(al)):
    ss = 0
    for j in range(len(al)):
        if i == j:
            continue
        else:
            ss += DHFEs_support(al[i],al[j],3,2)        ## 使用欧几里得距离
    weig.append(weight[i]*(1+ss)/s)